In [1]:
from IPython.core.interactiveshell import InteractiveShell  #执行该代码可以使得当前nb支持多输出
InteractiveShell.ast_node_interactivity = "all" 
import numpy as np
from datetime import datetime
import pandas as pd 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
pd.options.display.max_rows = 8  
%matplotlib inline

In [2]:
tmp = pd.read_excel('../data/taobao/new/shipin_plus.xlsx')

names = tmp['店铺名字']  #得到所有店铺的名称列表

In [5]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import re
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from snownlp import SnowNLP
import jieba.analyse as ana
import jieba
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer


sen_avg_list= []  #综合情感值得分列表
grades_list = [] #负面评论所占比重列表

for parent,dirnames,filenames in os.walk(r'../data/taobao/new/评论数据_食品/'):
    for filename in filenames:
        #root 所指的是当前正在遍历的这个文件夹的本身的地址
        #dirs 是一个 list ，内容是该文件夹中所有的目录的名字(不包括子目录)
        #files 同样是 list , 内容是该文件夹中所有的文件(不包括子目录)

#         print(parent+filename)
        df = pd.read_csv(parent+filename,skiprows =1,lineterminator="\n") 
        df = df.iloc[:len(df)-2]['评价内容']
#         print(df)
        
        #分词
        dict1 = '../data/nlp/电商客服输入法.txt'
        dict2 = '../data/nlp/电子商务专用词汇.txt'
        jieba.load_userdict(dict1) # dict_为自定义词典的路径
        jieba.load_userdict(dict2) # dict_为自定义词典的路径



        # 去除停用词
        # 使用预先准备的停用词表
#         tmpdf = pd.read_csv('../data/nlp/停用词.txt',
#                             names = ['w'], sep = 'aaa', encoding = 'utf-8')
#         tmpdf.head()

        ana.set_stop_words('../data/nlp/停用词.txt')

        for i,value in enumerate(df):
            res = ana.extract_tags(value , topK = 50) # 使用TF-IDF算法提取关键词，并同时去掉停用词
            df[i] = ' '.join(res)
  
        
    
        #特征提取
        vectorizer = CountVectorizer() 
        X = vectorizer.fit_transform(df) # 将文本中的词语转换为词频矩阵  
        transformer = TfidfTransformer()  
        tfidf = transformer.fit_transform(X)  #基于词频矩阵X计算TF-IDF值  
        tfidf


#         tfidf.toarray() # 转换为数组
#         tfidf.todense() # 转换为矩阵
#         tfidf.todense().shape
#         print("字典长度：", len(vectorizer.vocabulary_))
#         vectorizer.vocabulary_
        
        #抽取文档主题
        # 设定LDA模型
        n_topics = 10
        ldamodel = LatentDirichletAllocation(n_components = n_topics)
        
        # 拟合LDA模型
        ldamodel.fit(tfidf)
        
        # 拟合后模型的实质
#         print(ldamodel.components_.shape)
#         ldamodel.components_[:2]

        
        #打印出前十个主题的10关键词信息
        n_top_words = 10
        tf_feature_names = vectorizer.get_feature_names()

        #--------------------------------------------------------------------------------------
        #获取每家店铺的情感值得分  
        sen_total = []
        for topic_idx, topic in enumerate(ldamodel.components_):
            sum_ = 0
            str_topic = np.array([tf_feature_names[i] 
                            for i in topic.argsort()[:-n_top_words - 1:-1]])
            if len(str_topic) > 0 :
                for i in str_topic:     #对每个主题的情绪值求和
                   sum_ += SnowNLP(i).sentiments

            else:
                sum_ = 0
            sen_total.append(sum_)

        sen_total = np.array(sen_total)
        sen_avg = sen_total.mean()  #对各个主题求平均
        sen_avg_list.append(sen_avg)   
        #--------------------------------------------------------------------------------------
        
        
        #--------------------------------------------------------------------------------------
        #获取负面评论
        grades = []
        neg_grades = []
        try:
            for i, value in enumerate(df):
                sum_ = 0 
                for j in value.split(" "):
                    sum_+= SnowNLP(j).sentiments
                avg_word = sum_/len(value.split(" "))  
                
                
                if avg_word < 0.5:
                    neg_grades.append(avg_word)
                grades.append(avg_word)    
           
        except:
            avg_word = 0
            neg_grades.append(avg_word)
            grades.append(avg_word)    
            pass

       
        # grades        
        ratio =len( neg_grades)/len(grades)
        grades_list.append(ratio)  
        #--------------------------------------------------------------------------------------
        

sen_avg_list
grades_list


to_file = pd.DataFrame(sen_avg_list)
to_file['负面评论比重'] = grades_list
to_file['店铺名称'] = names
to_file.to_csv(r'../data/taobao/new/nlp特征_食品.csv',encoding="utf_8_sig")

<1454x450 sparse matrix of type '<class 'numpy.float64'>'
	with 4615 stored elements in Compressed Sparse Row format>

LatentDirichletAllocation()

<2000x1765 sparse matrix of type '<class 'numpy.float64'>'
	with 13490 stored elements in Compressed Sparse Row format>

LatentDirichletAllocation()

<2000x1388 sparse matrix of type '<class 'numpy.float64'>'
	with 7725 stored elements in Compressed Sparse Row format>

...

LatentDirichletAllocation()

<2000x683 sparse matrix of type '<class 'numpy.float64'>'
	with 6078 stored elements in Compressed Sparse Row format>

LatentDirichletAllocation()

[5.465662443144791,
 6.02829678732853,
 5.899148894736737,
 5.668299383276963,
 5.9275218489035355,
 5.7869599686052275,
 6.101946524597176,
 5.776401046266082,
 6.085968003414379,
 5.667816583066816,
 6.154916342496824,
 5.507999650531569,
 5.400682810136027,
 6.0624560633799245,
 5.5744373490401085,
 6.120014504983384,
 5.799516412069044,
 6.023437675643645,
 6.0891492604330235,
 6.203926220891448,
 6.076549995486869,
 5.742113618234688,
 5.727930790745396,
 5.536053503872047,
 5.726148497233404,
 5.430360150643047,
 5.604252463619858,
 6.286031137719798,
 5.709972859571843,
 6.033751350065529,
 5.856415922376381,
 5.704435058422798,
 5.701605232757169,
 6.226043693879134,
 5.966570442419693,
 6.277198737815218,
 5.822899553503156,
 5.885788199262544,
 5.785606579828281,
 5.924032891264985,
 5.426686126056774,
 6.156093270222317,
 5.328199130986725,
 5.8799932769495245,
 5.809145206106507,
 5.67141982347416,
 5.818306835266513,
 5.904672368662091,
 5.4957311480831565,
 5.463159413475

[0.25,
 0.1248630887185104,
 0.14814814814814814,
 0.19436997319034852,
 0.09868421052631579,
 0.19047619047619047,
 0.16952789699570817,
 0.1307471264367816,
 0.21605839416058395,
 0.31266149870801035,
 0.3093220338983051,
 0.32751091703056767,
 0.09090909090909091,
 0.19607843137254902,
 0.2608695652173913,
 0.1744186046511628,
 0.20765027322404372,
 0.1650485436893204,
 0.2,
 0.2,
 0.1956521739130435,
 0.31902985074626866,
 0.25882352941176473,
 0.296551724137931,
 0.2692307692307692,
 0.1917808219178082,
 0.37142857142857144,
 0.1875,
 0.2897196261682243,
 0.13414634146341464,
 0.2751677852348993,
 0.21428571428571427,
 0.17796610169491525,
 0.15774647887323945,
 0.14878048780487804,
 0.22388059701492538,
 0.22030651340996169,
 0.45161290322580644,
 0.2376237623762376,
 0.13725490196078433,
 0.45454545454545453,
 0.13513513513513514,
 0.36538461538461536,
 0.17616580310880828,
 0.15714285714285714,
 0.25609756097560976,
 0.2222222222222222,
 0.17073170731707318,
 0.1956521739130435

In [ ]:
tmp = pd.read_excel(r'../data/taobao/new/shipin_plus.xlsx')


file_all2 = pd.DataFrame()
file_all2['词频综合'] = 0
file_all2['店铺名称'] = tmp['店铺名字']

for parent,dirnames,filenames in os.walk(r'../data/taobao/new/评论数据_食品/'):
    for filename in filenames:
        
#         print(filename[0:-4])
#         print(parent+filename)
        df = pd.read_csv(parent+filename,skiprows =1,lineterminator="\n") 
        df = df.iloc[:len(df)-2]['评价内容']
        
        #分词
        dict1 = '../data/nlp/电商客服输入法.txt'
        dict2 = '../data/nlp/电子商务专用词汇.txt'
        jieba.load_userdict(dict1) # dict_为自定义词典的路径
        jieba.load_userdict(dict2) # dict_为自定义词典的路径


        # 去除停用词
        ana.set_stop_words('../data/nlp/停用词.txt')

        for i,value in enumerate(df):
            res = ana.extract_tags(value , topK = 50) # 使用TF-IDF算法提取关键词，并同时去掉停用词
            df[i] = ' '.join(res)
                         
        str_sum = ""  #把一篇文章的所有评论综合在一起
        for i in df:
            str_sum += i
#         print(str_sum)
  
#         file_all[file_all['店铺名称'] == filename[0:-4]]['词频综合得分'] = str_sum
        for i,filename2 in enumerate(file_all2['店铺名称']):
            if filename2.strip() == filename[0:-4].strip():
                file_all2['词频综合'][i] = str_sum   
                
        

#         print(file_all[file_all['店铺名称'] == filename]['词频综合得分'])
file_all2.to_csv(r'../data/taobao/new/nlp词频综合_食品.csv',encoding="utf_8_sig")
        
#         print(df)

In [ ]:
from sklearn.cluster import DBSCAN
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

db = DBSCAN(eps=1.1,min_samples=2, n_jobs=4).fit(data)

r = pd.Series(db.labels_, index =file_all.index)   # 详细输出每个样本对应的类别

r.to_excel(r'../data/taobao/new/data_type2.xls')
print(r.sort_values())

In [ ]:
final1 = pd.read_csv(r'../data/taobao/new/nlp特征.csv')
final1
final2 = pd.read_excel(r'../data/taobao/new/data_type2.xls')
final2

flag = pd.read_excel(r'../data/taobao/new/dianqi_plus.xlsx')
flag

final = pd.merge(final1,final2,how='inner',left_on='店铺名称',right_on='店铺名称')
final.corr()
flag = pd.merge(flag,final,how='inner',left_on='店铺名字',right_on='店铺名称')
flag.to_csv(r'../data/taobao/new/nlp特征最终_new.csv',encoding="utf_8_sig")